# Hi :)

----

# Import libraries

In [1]:
import pandas as pd
import numpy as np

# Dataset

Reading the dataset

In [2]:
dataset= pd.read_excel('Dataset/Data.xlsx')
dataset.head()

,Sensor 1,Sensor 2,Sensor 3,Real data
0,1.290892,1.233738,0.804856,1.20000
1,1.258611,0.693333,1.345625,1.08000
2,0.885958,0.924730,0.924650,0.97200
3,1.289685,1.244266,0.971379,0.87480
4,0.862174,0.822418,0.761150,0.78732


Checking the dataset

In [3]:
# checking the dataset null values
dataset.isna().sum()

Sensor 1     0
Sensor 2     0
Sensor 3     0
Real data    0
dtype: int64

In [4]:
# checking the dataset columns data type
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Sensor 1   1200 non-null   float64
 1   Sensor 2   1200 non-null   float64
 2   Sensor 3   1200 non-null   float64
 3   Real data  1200 non-null   float64
dtypes: float64(4)
memory usage: 37.6 KB


In [5]:
# checking the dataset statistical informations
dataset.describe()

,Sensor 1,Sensor 2,Sensor 3,Real data
count,1200.000000,1200.000000,1200.000000,1200.000000
mean,0.916810,0.915656,0.907631,0.915172
std,0.365527,0.365502,0.374501,0.244214
min,-0.477581,-0.389440,-0.579880,0.200126
25%,0.674057,0.684622,0.671952,0.739000
50%,0.937841,0.931848,0.935391,0.956681
75%,1.162877,1.157810,1.155175,1.116139
max,2.246458,2.609893,2.187203,1.341588


Lets make a copy of original dataset so we can work on it

In [6]:
df= dataset.copy()

# Ranking the sensors

Calculating the error for each sensor and calculating the matrics for each sensor

In [7]:
# Extract sensor readings and real values
sensors= ["Sensor 1", "Sensor 2", "Sensor 3"]
real_values= df["Real data"]

# Compute errors for each sensor
errors= {sensor : df[sensor] - real_values for sensor in sensors}

# Calculate error metrics
error_metrics= {
    sensor : {
        "MAE" : np.mean(np.abs(errors[sensor])),
        "MSE" : np.mean(errors[sensor] ** 2),
        "Variance" : np.var(errors[sensor])
    }
    for sensor in sensors
}

# Convert to DataFrame for better readability
error_df= pd.DataFrame(error_metrics).T

error_df

,MAE,MSE,Variance
Sensor 1,0.203822,0.078307,0.078304
Sensor 2,0.205767,0.078114,0.078114
Sensor 3,0.209327,0.078888,0.078831


Based on the matric and error_df, it seams that the ordering from 1 to 3 is the best ordering, so we don't change it

In [8]:
error_df

,MAE,MSE,Variance
Sensor 1,0.203822,0.078307,0.078304
Sensor 2,0.205767,0.078114,0.078114
Sensor 3,0.209327,0.078888,0.078831


# OWA

Optimistic & pesimistic

In [9]:
# Define OWA weight methods
def get_owa_weights(method):
    if method == "Optimistic":
        return np.array([1, 0, 0])
    elif method == "Pessimistic":
        return np.array([0, 0, 1])
    else:
        raise ValueError("Unknown method")

# Function to compute OWA output
def compute_owa(df, method):
    weights= get_owa_weights(method)
    owa_values= []

    for _, row in df[sensors].iterrows():
        owa_result= np.dot(weights, row)
        owa_values.append(owa_result)
    
    return np.array(owa_values)

# Apply OWA for each method
owa_results= {}
for method in ["Optimistic", "Pessimistic"]:
    owa_results[method]= compute_owa(df, method)

# Add OWA results to DataFrame
for method in owa_results:
    df[f"OWA_{method}"]= owa_results[method]

df.head()

,Sensor 1,Sensor 2,Sensor 3,Real data,OWA_Optimistic,OWA_Pessimistic
0,1.290892,1.233738,0.804856,1.20000,1.290892,0.804856
1,1.258611,0.693333,1.345625,1.08000,1.258611,1.345625
2,0.885958,0.924730,0.924650,0.97200,0.885958,0.924650
3,1.289685,1.244266,0.971379,0.87480,1.289685,0.971379
4,0.862174,0.822418,0.761150,0.78732,0.862174,0.761150


Calculating matrics for OWA outputs

In [10]:
# Compute MAE and MSE for each OWA method
owa_errors= {}
for method in ["Optimistic", "Pessimistic"]:
    mae= np.mean(np.abs(df[f"OWA_{method}"] - df["Real data"]))
    mse= np.mean((df[f"OWA_{method}"] - df["Real data"]) ** 2)
    owa_errors[method] = {"MAE" : mae, "MSE" : mse}

# Convert results to a DataFrame
owa_error_df= pd.DataFrame(owa_errors).T

owa_error_df

,MAE,MSE
Optimistic,0.203822,0.078307
Pessimistic,0.209327,0.078888


Calculating Orness & Dispersion

In [11]:
def compute_orness(weights):
    n= len(weights)
    return np.sum([(n - i) * weights[i - 1] for i in range(1, n + 1)]) / (n - 1)

def compute_dispersion(weights):
    return -np.sum([w * np.log(w) if w != 0 else 0 for w in weights])  # Avoid log(0)

# Compute for each method
owa_properties= {}
for method in ["Optimistic", "Pessimistic"]:
    weights= get_owa_weights(method)
    orness= compute_orness(weights)
    dispersion= compute_dispersion(weights)
    owa_properties[method]= {"Orness" : orness, "Dispersion" : dispersion}

# Convert to DataFrame
owa_properties_df= pd.DataFrame(owa_properties).T

owa_properties_df

,Orness,Dispersion
Optimistic,1.0,-0.0
Pessimistic,0.0,-0.0


Learning method

In [12]:
# initialize OWA weights randomly and normalize them
n= len(sensors)
np.random.seed(48)
weights= np.random.rand(n)
weights /= np.sum(weights)  # Normalize to sum to 1

# Learning parameters
learning_rate= 0.01
epochs= 1000

# Gradient Descent Optimization
for epoch in range(epochs):
    # Compute OWA output with current weights
    owa_output= np.dot(df[sensors], weights)

    # Compute error
    error= real_values - owa_output
    gradient= -2 * np.dot(df[sensors].T, error) / len(real_values)

    # Update weights
    weights -= learning_rate * gradient
    weights= np.maximum(weights, 0)  # Ensure non-negative
    weights /= np.sum(weights)  # Normalize

# Step 4: Apply OWA with learned weights
owa_result= np.dot(df[sensors], weights)

# Display final learned weights and first few results
print("Learned OWA Weights:", weights)
print("First few OWA Results:", owa_result[ : 5])

df['OWA_Learning']= owa_result
df.head()

Learned OWA Weights: [0.0786207  0.69268036 0.22869894]
First few OWA Results: [1.14014677 0.88695407 0.92166362 1.18542801 0.81153184]


,Sensor 1,Sensor 2,Sensor 3,Real data,OWA_Optimistic,OWA_Pessimistic,OWA_Learning
0,1.290892,1.233738,0.804856,1.20000,1.290892,0.804856,1.140147
1,1.258611,0.693333,1.345625,1.08000,1.258611,1.345625,0.886954
2,0.885958,0.924730,0.924650,0.97200,0.885958,0.924650,0.921664
3,1.289685,1.244266,0.971379,0.87480,1.289685,0.971379,1.185428
4,0.862174,0.822418,0.761150,0.78732,0.862174,0.761150,0.811532


Calculating matrics and Orness and Dispersion for Learning method

In [13]:
# Compute MAE & MSE
mae_owa= np.mean(np.abs(owa_result - real_values))
mse_owa= np.mean((owa_result - real_values) ** 2)

# Compute Orness
def compute_orness(weights):
    n= len(weights)
    return np.sum([(n - i) * weights[i - 1] for i in range(1, n + 1)]) / (n - 1)

# Compute Dispersion
def compute_dispersion(weights):
    return -np.sum([w * np.log(w) if w > 0 else 0 for w in weights])  # Avoid log(0)

orness= compute_orness(weights)
dispersion= compute_dispersion(weights)

# Display results
print(f"MAE of OWA: {mae_owa:.4f}")
print(f"MSE of OWA: {mse_owa:.4f}")
print(f"Orness: {orness:.4f}")
print(f"Dispersion: {dispersion:.4f}")

MAE of OWA: 0.1552
MSE of OWA: 0.0426
Orness: 0.4250
Dispersion: 0.7917
